### Setting Up Conda Environment with Python 3.12 and Installing Required Libraries
##### Step 1 - Created a Conda Environment with Python 3.12 and Connected to its Kernel
I created a new Conda environment with Python 3.12 and activated it. After that, I connected to the Python kernel in Jupyter/VSCode using this environment.
Commands to create and activate the environment:
```bash
conda create -n llm_env python=3.12
conda activate llm_env
```

##### Step 2 - Imported Required Libraries
Once the environment was set up and activated, I imported the following libraries for my project:
- `numpy` as `np`: For numerical operations and working with arrays.
- `pandas` as `pd`: For data manipulation and analysis.
- `torch`: PyTorch, for deep learning model building and training.
- `tqdm.notebook`: For displaying progress bars in Jupyter notebooks.
- `sklearn.preprocessing.LabelEncoder`: For encoding categorical labels into numerical format.
- `sklearn.model_selection.train_test_split`: For splitting datasets into training and testing sets.
- `transformers.BertTokenizer`: For tokenizing text data for BERT models.
- `transformers.BertForSequenceClassification`: For using BERT for sequence classification tasks.
- `torch.utils.data.TensorDataset`: For creating datasets from tensors.
- `torch.utils.data.DataLoader`: For batching datasets and loading them efficiently during training.
- `torch.utils.data.RandomSampler` and `torch.utils.data.SequentialSampler`: For random and sequential sampling of datasets.
- `transformers.AdamW` and `transformers.get_linear_schedule_with_warmup`: For the AdamW optimizer and learning rate scheduler used with transformers.
- `sklearn.metrics.f1_score`: For evaluating model performance using the F1 score.
- `random`: For random number generation, typically used in data shuffling.

Commands to install the libraries:

```bash
conda install numpy pandas scikit-learn tqdm pytorch
conda install -c huggingface transformers

In [31]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
import random